# ATENÇÃO

PARA FUNCIONAMENTO COMPLETO DO CÓDIGO, É NECESSÁRIO RODAR CADA CÉLULA PASSO A PASSO, RESPEITANDO O SEU TEMPO DE EXECUÇÃO. OU SEJA, RODE A PRÓXIMA CÉLULA SOMENTE QUANDO A CÉLULA ATUAL TERMINAR DE EXECUTAR.

O TEMPO DE EXECUÇÃO COSTUMA VARIA DE AMBIENTE PARA AMBIENTE E VELOCIDADE DE INTERNET, JUSTAMENTE POR CADA ARQUIVO A SER LIDO E EXECUTADO TER MAIS DE 1.5GB DE DADOS.


# Etapa 1  - Extração dos dados

Esta etapa do código é responsável por automatizar o processo de download dos arquivos CSVs contendo os dados do Cadastro Nacional da Pessoa Jurídica (CNPJ) do site dados.gov.br. Obs: Os arquivos serão todos baixados localmente, com o tempo de download variando de acordo com a velocidade da internet utilizada.

OBS 1: 

1- Não esquecer de setar o localhost do client corretamente para fazer a conexão com o banco
    
                client = pymongo.MongoClient("mongodb://localhost:27017/")
                
2- Não esquecer de setar o path do local de onde foi realizado download dos arquivos
    
                arquivos = r"C:\Users\louis\Downloads"

OBS 2: Será necessário esperar realizar todos os downloads, você pode ir acompanhando o status através do próprio webdriver. Nota-se que serão baixados 9 ( nove ) arquivos no total, onde em média cada um tem em torno de 1.5GB a 3GB de dados. 
O download será realizado de acordo com a velocidade de internet e capacidade do computador.

In [16]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By


options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(executable_path='C:\chromedriver')


driver.get("https://dados.gov.br/dados/conjuntos-dados/cadastro-nacional-da-pessoa-juridica---cnpj")
time.sleep(2)
driver.find_element("xpath", '/html/body/div/section/div/div[3]/div[2]/div[3]/div[2]').click()

resources = [resource.find_element("xpath", './/button[@id="btnDownloadUrl"]') for resource in 
             driver.find_elements("xpath", '//*[@id="collapse-recursos"]//div[@class="row flex mb-5"]') 
             if "Estabelecimento" in resource.text ]

for button in resources:
    time.sleep(1)
    button.click()
driver.close()

print("Finished running")

C:\Users\louis\AppData\Local\Temp\ipykernel_17360\1264683989.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:\chromedriver')


Finished running


# Etapa 2  - Extração dos dados

Realizar armazenamento dos dados extraídos na fonte no MongoBD

OBS: 

1- Não esquecer de setar o localhost do client corretamente para fazer a conexão com o banco
    
                client = pymongo.MongoClient("mongodb://localhost:27017/")
                
2- Não esquecer de setar o path do local de onde foi realizado download dos arquivos
    
                arquivos = r"C:\Users\louis\Downloads"

In [19]:
#write mongodb
import csv
import time
import pymongo
import os
from dask import dataframe as dd
from collections import defaultdict
import dask.bag as db
import dask_mongo
import pandas as pd
import zipfile
import json
from datetime import datetime
import dateutil.parser

def dateparse(s):
    try:
        return datetime.strptime(s, '%Y%m%d')
    except ValueError:
        return datetime.min
    
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["estabelecimentos"]
mycol = mydb["estabelecimentos"]

#hardcoded
path = r"C:\Users\louis\Downloads"

#range hardcoded alterar
for i in range(10):
    file = os.path.join(path, f"Estabelecimentos{i}.zip")
    zf = zipfile.ZipFile(file) 
    filename = zf.namelist()[0]
    chunksize = 10 ** 4
    names = ["CNPJ_BASICO", "CNPJ_ORDEM", "CNPJ_DV", "IDENTIFICADOR", "FANTASIA", "SIT_CADASTRAL", "DATA_SIT_CADASTRAL", "MOT_SIT_CADASTRAL", "NOME_EXT", "PAIS", "DATA_INI", "CNAE_PRINCIPAL", "CNAE_SECUNDARIA", "TIPO_LOGRADOURO", "LOGRADOURO", "NUMERO", "COMPLEMENTO", "BAIRRO", "CEP", "UF", "MUNICIPIO", "DDD1", "TEL1", "DDD2", "TEL2", "DDD_FAX", "FAX", "EMAIL", "SIT_ESP","DATA_ST_ESP"]
    date_cols = ['DATA_SIT_CADASTRAL', 'DATA_INI']

    print(f"Estabelecimentos{i}")
    
    start = time.time()

    with pd.read_csv(zf.open(filename), dtype = str, date_parser=dateparse, parse_dates=date_cols, chunksize = chunksize, names = names, header = None, encoding="ISO-8859-1", delimiter=';', keep_default_na=False) as reader:
        for chunk in reader:
            #start_chunk = time.time()
            #chunk['DATA_ST_ESP']= chunk['DATA_ST_ESP'].where(pd.isnull(chunk['DATA_ST_ESP']), datetime.min)
            mycol.insert_many(chunk.to_dict(orient='records'))
            print('.', end = '')
            


    print("\nTotal Write Time on mongo: ",(time.time()-start),"sec")
    print("Finished running")

Estabelecimentos0
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# ETAPA 3 -  Realização das consultas no banco, transformações das consultas em CSV e correlação de tabela.

Nesse código, ocorre a realização das consultas dos dados inseridos no banco, além de que, para cada consulta realizada, serão salvos os dados em arquivos no formato CSV.

In [1]:
import pymongo
import time
from geopy.geocoders import Nominatim
import geopy.distance
import pandas as pd
import zipfile
import re
import functools

print("Finished running")

Finished running


# 4.1 - getActiveEnterprisesPercentage(collection)

Essa função calcula a porcentagem de empresas ativas em relação ao total de empresas no banco de dados. Ela utiliza a biblioteca pandas para manipular os dados e retorna um DataFrame contendo o resultado.

In [2]:
# 4.1
print("Running")
def getActiveEnterprisesPercentage(collection):
    pipeline = [
        {
            "$facet": {
                "Total": [
                    {"$count": "Total"},
                ],
                "Active": [
                    {"$match": {"SIT_CADASTRAL": {"$in": ["2", "02"]}}},
                    {"$count": "Active"},
                ],
            }
        },
        {
            "$project": {
                "TOTAL": {"$arrayElemAt": ["$Total.Total", 0]},
                "ACTIVE": {"$arrayElemAt": ["$Active.Active", 0]},
            }
        },
    ]
    result = collection.aggregate(pipeline)
    result_dict = list(result)[0]
    result_dict["ACTIVE_PERCENTAGE"] = result_dict["ACTIVE"] / result_dict["TOTAL"]
    print("Mid")

    df = pd.DataFrame(result_dict, index=[0])
    return df


Running


# 4.2 - getRestaurantsByYear(collection)
Essa função retorna a quantidade de restaurantes registrados no banco de dados e agrupa eles de acordo por ano. Ela utiliza expressões regulares e a biblioteca pandas para processar os dados e retorna um DataFrame com o resultado.

Detecta os restaurantes de acordo com o prefixo, que é o 561

In [3]:
# 4.2
# 561 prefixo restaurante
print("Running")
def getRestaurantsByYear(collection):
    prefix = "561"
    pattern = re.compile(f"^{prefix}")
    pipeline = [
        {"$match": {"CNAE_PRINCIPAL": {"$regex": pattern}}},
        {
            "$group": {
                "_id": {"$year": "$DATA_INI"},
                "CNPJS": {
                    "$push": {
                        "$concat": [
                            {"$substr": ["$CNPJ_BASICO", 0, 2]},
                            ".",
                            {"$substr": ["$CNPJ_BASICO", 2, 3]},
                            ".",
                            {"$substr": ["$CNPJ_BASICO", 5, 3]},
                            "/",
                            "$CNPJ_ORDEM",
                            ".",
                            "$CNPJ_DV",
                        ]
                    }
                },
                "TOTAL": {"$sum": 1},
            }
        },
        {"$project": {"_id": 0, "YEAR": "$_id", "CNPJS": "$CNPJS", "TOTAL": "$TOTAL"}},
        {"$sort": {"YEAR": 1}},
    ]
    restaurants = list(collection.aggregate(pipeline))
    df = pd.DataFrame(restaurants)
    print("Finished running")
    return df

Running


# 4.3 - getDistinctAddress(collection, prefix)
Essa função recupera os endereços distintos dos estabelecimentos no banco de dados que correspondem a um determinado prefixo de CEP. Ela utiliza expressão regular e a biblioteca pandas para manipular os dados e retorna uma lista de objetos contendo os detalhes dos endereços.


In [4]:
# 4.3
print("Running")
def getDistinctAddress(collection, prefix):
    pattern = re.compile(f"^{prefix}")
    pipeline = [
        {"$match": {"CEP": {"$regex": pattern}}},
        {
            "$group": {
                "_id": "$CEP",
                "CNPJS": {
                    "$push": {
                        "$concat": [
                            {"$substr": ["$CNPJ_BASICO", 0, 2]},
                            ".",
                            {"$substr": ["$CNPJ_BASICO", 2, 3]},
                            ".",
                            {"$substr": ["$CNPJ_BASICO", 5, 3]},
                            "/",
                            "$CNPJ_ORDEM",
                            ".",
                            "$CNPJ_DV",
                        ]
                    }
                },
                "TIPO_LOGRADOURO": {"$first": "$TIPO_LOGRADOURO"},
                "LOGRADOURO": {"$first": "$LOGRADOURO"},
                "NUMERO": {"$first": "$NUMERO"},
                "BAIRRO": {"$first": "$BAIRRO"},
                "UF": {"$first": "$UF"},
                "TOTAL": {"$sum": 1},
            }
        },
    ]
    ceps = collection.aggregate(pipeline)
    print("Finished running")
    return ceps

Running


In [5]:
# 4.3
print("Running")
def getCnpjsByDistanceToOrigin(collection, ceps, locator, pre_coords={}):
    coords = {}
    coords_to_update = {}
    origin_address = "ALAMEDA FRANCA 139, JARDIM PAULISTA, SP"
    origin_coords = (-23.567769, -46.6568235)

    for cep in ceps:

        if cep["_id"] in pre_coords:
            # list(set(lst1) & set(lst2))
            coords[cep["_id"]] = (
                pre_coords[cep["_id"]][0],
                list(set(cep["CNPJS"]) & set(pre_coords[cep["_id"]][1])),
            )
            continue

        address = f"{cep['TIPO_LOGRADOURO']} {cep['LOGRADOURO']} {cep['NUMERO']} - {cep['BAIRRO']}, {cep['UF']}"

        location = locator.geocode(address)
        time.sleep(2) #because geopy API limits per second

        if location is None:
            continue

        point = location.point
        coord_aux = (point.latitude, point.longitude)
        distance = geopy.distance.geodesic(origin_coords, coord_aux).km
        if distance <= 5:
            coords[cep["_id"]] = (distance, cep["CNPJS"])
            coords_to_update[cep["_id"]] = coords[cep["_id"]]
       
    return (coords, coords_to_update)

Running


In [6]:
# 4.3
print("Running")
def getExistentCoords(collection):
    result = collection.find_one({})
    print("Finished running, existents coords")
    return result

Running


In [7]:
print("Running")
def updateCoords(collection, coords_to_update):
    if coords_to_update:
        collection.update_one({}, {"$set": coords_to_update}, upsert=True)
        print("Finished running, update coords")

Running


In [8]:
print("Running")
def insertCoords(collection, coords):
    collection.remove({})
    collection.insert_one(coords)
    print("Finished running, insert coords")

Running


# 4.4 - getCnaeCorrelation(collection)

Essa função calcula a correlação entre os códigos CNAE (Classificação Nacional de Atividades Econômicas) principais e secundários dos estabelecimentos. Ela utiliza a biblioteca pandas para manipular os dados e retorna um DataFrame com os resultados da correlação.

In [9]:
# 4.4
print("Running")
def getCnaeCorrelation(collection):
    pipeline = [
        {
            "$project": {
                "CNAE_PRINCIPAL": 1,
                "CNAE_SECUNDARIA": {"$split": ["$CNAE_SECUNDARIA", ","]},
            }
        },
        {"$unwind": "$CNAE_SECUNDARIA"},
        {
            "$group": {
                "_id": {
                    "CNAE_PRINCIPAL": "$CNAE_PRINCIPAL",
                    "CNAE_SECUNDARIA": "$CNAE_SECUNDARIA",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$group": {
                "_id": "$_id.CNAE_PRINCIPAL",
                "CNAE_SECUNDARIA_counts": {
                    "$push": {
                        "CNAE_SECUNDARIA": "$_id.CNAE_SECUNDARIA",
                        "count": "$count",
                    }
                },
            }
        },
        {
            "$project": {
                "_id": 0,
                "CNAE_PRINCIPAL": "$_id",
                "CNAE_SECUNDARIAS": {
                    "$slice": [
                        {
                            "$sortArray": {
                                "input": "$CNAE_SECUNDARIA_counts",
                                "sortBy": {"count": -1, "CNAE_SECUNDARIA": 1},
                            }
                        },
                        10,
                        #top 10
                    ]
                },
            }
        },
        {"$sort": {"CNAE_PRINCIPAL": 1}},
    ]

    result = list(collection.aggregate(pipeline))

    result = {
        document["CNAE_PRINCIPAL"]: {
            secundaria["CNAE_SECUNDARIA"]: secundaria["count"]
            for secundaria in document["CNAE_SECUNDARIAS"]
        }
        for document in result
    }

    df = pd.DataFrame(result).fillna(0)
    correlation = df.corr()
    print("Finished running")
    return correlation

Running


# ACESSO AO BANCO

o test-database, é um banco de teste mais leve que roda mais rápido, ao contrário do estabelecimentos, que é mais pesado

In [10]:
print("Running")
locator = Nominatim(user_agent="application")
client = pymongo.MongoClient("mongodb://localhost:27017/")
#mydb = client["test-database"]
#mycol = mydb["test-collection"]
mydb = client["estabelecimentos"]
mycol = mydb["estabelecimentos"]
print("Finished running")

Running
Finished running


# GERANDO CSV s

In [11]:
print("Running")
start = time.time()

# 4.1
df_active = getActiveEnterprisesPercentage(mycol)
print(df_active)
df_active.to_csv("empresas_ativas.csv")

Running
Mid
Finished running
      TOTAL    ACTIVE  ACTIVE_PERCENTAGE
0  56132011  23248829           0.414181


In [16]:
# 4.2
df_restaurants = getRestaurantsByYear(mycol)
print(df_restaurants)
df_restaurants.to_csv("restaurantes_por_ano.csv")


Finished running
    YEAR                                              CNPJS   TOTAL
0   1920                               [00.472.818/0001.07]       1
1   1932                               [69.000.321/0001.92]       1
2   1940                               [61.358.859/0001.61]       1
3   1949                               [60.765.468/0001.07]       1
4   1950           [60.432.671/0001.53, 33.212.903/0001.95]       2
..   ...                                                ...     ...
72  2019  [32.330.217/0001.56, 32.330.601/0001.59, 32.33...  226725
73  2020  [35.869.394/0001.01, 35.869.467/0001.57, 35.86...  253303
74  2021  [37.929.371/0002.43, 40.228.502/0001.99, 40.22...  278026
75  2022  [44.708.462/0001.33, 44.708.486/0001.92, 44.70...  228464
76  2023  [49.011.938/0001.04, 49.012.583/0001.60, 49.01...   59993

[77 rows x 3 columns]


In [18]:
# 4.3
pre_coords = getExistentCoords(client["estabelecimentos"]["coords"])
ceps = getDistinctAddress(mycol, "014")
coords, coords_to_update = getCnpjsByDistanceToOrigin(mycol, ceps, locator, pre_coords)
if not pre_coords:
    insertCoords(client["estabelecimentos"]["coords"], coords)
else:
    updateCoords(client["estabelecimentos"]["coords"], coords_to_update)
    # updateCoords is optional (but recommended)

coords = {key: (value[0], value[1], len(value[1])) for key, value in coords.items()}
total = functools.reduce(lambda x, value: x + value[2], coords.values(), 0)
print("a")

# generate total number
df_coords_basic = pd.DataFrame({"EMPRESAS_PROXIMAS": total}, index=[0])
print(df_coords_basic)
df_coords_basic.to_csv("empresas_proximas_basic.csv")


Finished running, existents coords
Finished running
a
   EMPRESAS_PROXIMAS
0              93960


In [19]:
# generate doc with all cnpjs by cep
df_coords = pd.DataFrame(coords, index=["distance", "cnpjs", "total"])
print(df_coords)
df_coords.to_csv("empresas_proximas_cnpj.csv")

                                                   01411900  \
distance                                            0.86952   
cnpjs     [21.692.098/0001.65, 48.247.449/0001.85, 36.04...   
total                                                    55   

                                                   01410000  \
distance                                           0.861612   
cnpjs     [74.274.671/0001.87, 43.941.045/0001.73, 10.33...   
total                                                   566   

                                          01455120  \
distance                                    3.8872   
cnpjs     [53.689.204/0001.48, 04.466.485/0001.64]   
total                                            2   

                                                   01448010  \
distance                                           2.211677   
cnpjs     [00.903.654/0001.25, 54.516.620/0001.07, 48.82...   
total                                                    42   

                        

In [20]:
# 4.4
print("inicio")
correlation_cnae = getCnaeCorrelation(mycol)
print(correlation_cnae)
correlation_cnae.to_csv("correlacao_cnae.csv")

end = time.time()
print("\nTotal Write Time on mongo: ", (end - start), "sec")

print("Finished running")


inicio
Finished running
          0111301   0111302   0111303   0111399   0112101   0112102   0112199  \
0111301  1.000000  0.665638  0.552213  0.947856  0.807087  0.803693  0.775428   
0111302  0.665638  1.000000  0.636877  0.701294  0.619185  0.381235  0.550348   
0111303  0.552213  0.636877  1.000000  0.469118  0.817519  0.307755  0.365496   
0111399  0.947856  0.701294  0.469118  1.000000  0.761138  0.770699  0.899823   
0112101  0.807087  0.619185  0.817519  0.761138  1.000000  0.654928  0.648456   
...           ...       ...       ...       ...       ...       ...       ...   
9609207  0.668320  0.265420  0.058992  0.621871  0.317244  0.578534  0.503956   
9609208  0.693233  0.275836  0.061772  0.645156  0.329467  0.600140  0.523009   
9609299  0.804425  0.321148  0.072869  0.748846  0.383122  0.696481  0.607444   
9700500  0.813159  0.325027  0.074097  0.757053  0.387579  0.704072  0.614240   
9900800  0.816451  0.326739  0.074839  0.760196  0.389449  0.706953  0.616930   

   